# viz

> Fill in a module description here

In [ ]:
#| default_exp viz

In [ ]:
#| export
from neev.engine import *
from graphviz import Digraph

In [ ]:
#| hide
!pwd

/Users/kaushik.sinha/ws_p/neev/nbs


In [ ]:
#| export
def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

In [ ]:
#| export
def draw_dot(root, 
             format='svg',# png | svg | ... 
             rankdir='LR'):# TB (top to bottom graph) | LR (left to right)
    """
    visualize the graph using dot
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})
    
    for n in nodes:
        dot.node(name=str(id(n)), label = "{ data %.4f | grad %.4f }" % (n.data, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))
    
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()